In [38]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### Which genre sells most tracks?

In [136]:
%%sql
WITH countryfilt AS (SELECT * FROM invoice_line
INNER JOIN invoice i on invoice_line.invoice_id = i.invoice_id
INNER JOIN customer c on c.customer_id=i.customer_id
WHERE c.country='USA')
SELECT g.name Genre,SUM(co.quantity) Sold_quantity,(CAST(COUNT(*) AS FLOAT)/(SELECT COUNT(*) FROM countryfilt))*100 Percentage_sales 
FROM countryfilt co
INNER JOIN track t ON t.track_id=co.track_id
INNER JOIN genre g ON g.genre_id=t.genre_id
GROUP BY 1 
ORDER BY 2 DESC
LIMIT 10

Done.


Genre,Sold_quantity,Percentage_sales
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


### Which employee generated most sales?

In [77]:
%%sql
SELECT e.employee_id,e.first_name||" "||e.last_name Name,ROUND(SUM(i.total),2) generated_sales,ROUND(SUM(i.total)/(SELECT SUM(total) FROM invoice),2)*100 percentage_sales_generated,e.hire_date,e.country FROM invoice i
LEFT JOIN customer c ON c.customer_id=i.customer_id
LEFT JOIN employee e ON e.employee_id=c.support_rep_id
GROUP BY 1
ORDER BY 3 DESC
LIMIT 10



Done.


employee_id,Name,generated_sales,percentage_sales_generated,hire_date,country
3,Jane Peacock,1731.51,37.0,2017-04-01 00:00:00,Canada
4,Margaret Park,1584.0,34.0,2017-05-03 00:00:00,Canada
5,Steve Johnson,1393.92,30.0,2017-10-17 00:00:00,Canada


In [66]:
%%sql
SELECT employee_id,title FROM employee

Done.


employee_id,title,country
1,General Manager,Canada
2,Sales Manager,Canada
3,Sales Support Agent,Canada
4,Sales Support Agent,Canada
5,Sales Support Agent,Canada
6,IT Manager,Canada
7,IT Staff,Canada
8,IT Staff,Canada


### Which country should the company focus on for increasing revenue?

#### Some of the parameters we can use to select a country are:
* Number of customers 
* Total Sales value
* Avg Sales per Customer
* Average Order Value

### PARAMETER 1 - NUMBER OF CUSTOMERS PER COUNTRY

In [140]:
%%sql
WITH custs AS (SELECT country,COUNT(DISTINCT(customer_id)) custnum
FROM customer
GROUP BY 1)
SELECT country,custnum,CASE
                        WHEN custnum=1 THEN 'other'
                        ELSE country
                        END AS country_cat
from custs
ORDER BY 2 DESC

Done.


country,custnum,country_cat
USA,13,USA
Canada,8,Canada
Brazil,5,Brazil
France,5,France
Germany,4,Germany
United Kingdom,3,United Kingdom
Czech Republic,2,Czech Republic
India,2,India
Portugal,2,Portugal
Argentina,1,other


#### Since there are many countries with just one customer, categorize them as 'Other' 

### PARAMETER 2 - TOTAL SALES BY COUNTRY

In [141]:
%%sql
SELECT c.country,SUM(i.total) total_sales FROM invoice i
INNER JOIN customer c ON c.customer_id=i.customer_id
GROUP BY c.country
ORDER BY 2 DESC

Done.


country,total_sales
USA,1040.4899999999998
Canada,535.5900000000001
Brazil,427.68000000000006
France,389.0699999999999
Germany,334.62
Czech Republic,273.24000000000007
United Kingdom,245.52
Portugal,185.13000000000002
India,183.14999999999998
Ireland,114.83999999999997


### PARAMETER 3 - AVG CUSTOMER VALUE

In [139]:
%%sql
WITH per_customer AS (SELECT country,SUM(i.total) value_per_cust
FROM customer c
INNER JOIN invoice i ON i.customer_id=c.customer_id
GROUP BY c.customer_id)
SELECT country,ROUND(AVG(value_per_cust),2) avg_cust_value
FROM per_customer
GROUP BY 1
ORDER BY 2 DESC

Done.


country,avg_cust_value
Czech Republic,136.62
Ireland,114.84
Spain,98.01
Chile,97.02
Portugal,92.56
India,91.57
Brazil,85.54
Germany,83.66
United Kingdom,81.84
Australia,81.18


### PARAMETER 4 - AVG ORDER VALUE

In [122]:
%%sql
SELECT country,AVG(i.total) avg_order
FROM customer c
INNER JOIN invoice i ON i.customer_id=c.customer_id
GROUP BY c.country

Done.


country,avg_order
Argentina,7.92
Australia,8.118
Austria,7.699999999999999
Belgium,8.627142857142855
Brazil,7.0111475409836075
Canada,7.047236842105265
Chile,7.463076923076924
Czech Republic,9.108000000000002
Denmark,3.761999999999999
Finland,7.2


#### Here is a combined table representing the whole picture

In [143]:
%%sql
WITH t1 AS(WITH custs AS (SELECT country,COUNT(DISTINCT(customer_id)) custnum
FROM customer
GROUP BY 1)
SELECT country,custnum,CASE
                        WHEN custnum=1 THEN 'other'
                        ELSE country
                        END AS country_cat,CASE when custnum=1 THEN 1 ELSE 0 END AS sorter
from custs),t2 AS
(SELECT c.country,SUM(i.total) total_sales FROM invoice i
INNER JOIN customer c ON c.customer_id=i.customer_id
GROUP BY c.country),t3 AS
(WITH per AS (SELECT country,SUM(i.total) value_per_cust
FROM customer c
INNER JOIN invoice i ON i.customer_id=c.customer_id
GROUP BY c.customer_id)
SELECT country,AVG(value_per_cust) avg_cust_value
FROM per
GROUP BY 1),t4 AS
(SELECT country,AVG(i.total) avg_order
FROM customer c
INNER JOIN invoice i ON i.customer_id=c.customer_id
GROUP BY c.country)
SELECT t1.country_cat Country,SUM(t1.custnum) Num_custs,ROUND(SUM(t2.total_sales),2) Total_Sales,ROUND(AVG(t3.avg_cust_value),2) Avg_cust_value,ROUND(AVG(t4.avg_order),2) Avg_order_value  
FROM t1
LEFT JOIN t2 ON t2.country=t1.country
LEFT JOIN t3 ON t3.country=t1.country
LEFT JOIN t4 ON t4.country=t1.country
GROUP BY 1
ORDER BY t1.sorter,3 DESC

Done.


Country,Num_custs,Total_Sales,Avg_cust_value,Avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.56,6.38
India,2,183.15,91.57,8.72
other,15,1094.94,73.0,7.45


#### From the table, it is evident that Czech Republic has high Average Customer and Order values 